In [1]:
#=============================================================================#
#                                                                             #
# The MIT License (MIT)                                                       #
#                                                                             #
# Copyright (c) 2022 Maxwell A. Fine                                          #
#                                                                             #
# Permission is hereby granted, free of charge, to any person obtaining a     #
# copy of this software and associated documentation files (the "Software"),  #
# to deal in the Software without restriction, including without limitation   #
# the rights to use, copy, modify, merge, publish, distribute, sublicense,    #
# and/or sell copies of the Software, and to permit persons to whom the       #
# Software is furnished to do so, subject to the following conditions:        #
#                                                                             #
# The above copyright notice and this permission notice shall be included in  #
# all copies or substantial portions of the Software.                         #
#                                                                             #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR  #
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,    #
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE #
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER      #
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING     #
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER         #
# DEALINGS IN THE SOFTWARE.                                                   #
#                                                                             #
#=============================================================================#
# imports
import numpy as np
import matplotlib as pyplot
from astropy.io import fits


import os, sys, time, shutil
import astropy.io.fits as pyfits
import numpy as np
import re
#from swiftmonitor.utils import execute_cmd, region

import subprocess # for running shell commands

#-------------------------------------------------------------------------------
# arguments
results = '../results' # maybe I'll save stuff in the corresponding swift id dir? that seems reasonable
file_ext = '.pdf' # for the outputted plots
data_dir = '../data' # will loop through every dir for the bat data

# args
clobber = True # if true remakes/overwrites files
time_res = 1 # seconds?
energy_bins = 14-175
source_id = '00010374156'
evt_file = 'sw00010374156bevshsl_uf.evt.gz'


events = '/bat/event'
event_dir = data_dir + '/' + source_id + events
hk_dir = data_dir + '/' +source_id + '/bat/hk'
results_dir = results + '/' + source_id
#-----------------------------------------------------------------------------#
def get_swift_ids_data(data_dir):
    '''make list of swift IDS from the data dir'''
    output = subprocess.check_output(["ls",],universal_newlines=True, cwd= data_dir)
    source_ids = output.split() # list of strings the source IDs

    # pop off these dirs if they exist in the list
    source_ids.remove('GUANO')
    source_ids.remove('dump')
    source_ids.remove('inventory')
    source_ids.remove('-')
    source_ids.remove('GUANO')
    source_ids.remove('triggers.tsv')
    source_ids.remove('skycoords.dat')
    return source_ids
#-----------------------------------------------------------------------------#
def get_evt_files(event_dir):
    'makes a list of the evt files in a swift bat dir'
    bat_event_data = subprocess.check_output(["ls",],universal_newlines=True,
    cwd= event_dir)
    evt_files = bat_event_data.split()
    return evt_files
#-----------------------------------------------------------------------------#
def get_dir_trees(source_ids,data_dir,results):
    '''makes dicts of event, hk, and results dirs'''

    events = '/bat/event'

    event_dir = {}
    hk_dir = {}
    results_dir= {}
    for i in source_ids:
        source_id = i
        event_dir[i] = data_dir + '/' + source_id + events
        hk_dir[i] = data_dir + '/' +source_id + '/bat/hk'
        results_dir[i] = results + '/' + source_id

    return event_dir, hk_dir, results_dir
#-----------------------------------------------------------------------------#
def run_heasoft(source_id, evt_file, results, event_dir, hk_dir,
                results_dir, clobber=clobber, energy_bins=14-175, time_res=1):
    '''Wrapper function that runs the main heasoft commands, and produces results

    Args:
    '''

    j = evt_file

    # make entry in results dir based on swift id
    subprocess.run(['mkdir ' + str(source_id)], cwd= results, shell=True) # change 0 to i when making wrapper!!

    # make DPI
    subprocess.run(['batbinevt ' + str(j) + ' weighted=no '
    + 'outunits=counts outtype=dpi energy=-'
    +' clobber=' + str(clobber)
    +' outfile= frb.dpi'] , cwd = event_dir, shell=True)

    # move frb.dpi and (copy) det mask, event file. attitude file into results dir
    subprocess.run(['mv ' +str(event_dir) + '/frb.dpi '
                     + str(results_dir) + '/frb.dpi'  ],
                    shell=True, cwd = '../data')

    subprocess.run(['cp ' +str(hk_dir) + '/sw' + str(source_id) + 'bdecb.hk.gz '
                   + str(results_dir) + '/sw' + str(source_id) + 'bdecb.hk.gz'  ],
                   shell=True, cwd = '../data')

    subprocess.run(['cp data/' +str(source_id) + '/auxil/sw' + str(source_id) + 'sat.fits.gz '
                   + 'data/' + str(results_dir) + '/sw' + str(source_id) + 'sat.fits.gz' ],
                   shell=True, cwd = '..')

    subprocess.run(['cp data/' +str(source_id) + '/bat/event/' + str(j) + ' '
                   + 'data/' + str(results_dir) + '/' + str(j) ],
                   shell=True, cwd = '..')

    # make mask
    subprocess.run(['bathotpix detmask=sw' + str(source_id)
                    + 'bdecb.hk.gz outfile = frb.mask clobber =' + str(clobber) ],
                   shell=True, cwd=results_dir)

    # make lc with heasoft
    # should I set energy bins here and time res?
    subprocess.run(['batbinevt detmask=frb.mask ' + str(j)
                + ' timedel = ' + str(time_res)  + ' weighted=no outtype=lc energybins=- outfile =frb.heasoft.lc clobber ='
                + str(clobber) ],
                   shell=True, cwd=results_dir)

   # make frb2_dpi
   # this one should contain our target source
   # need to add target_time and time size stuff
   # I am unsure if the timedel is correct here
    subprocess.run(['batbinevt '  + str(j)
                + ' outfile =frb2.dpi energybins=' +str(energy_bins)
                + ' clobber= ' + str(clobber) + ' outtype=dpi detmask=frb.mask timedel = 0'
                + ' tstart=639072334.000000 tstop=639072896.800600' ],
               shell=True, cwd=results_dir)

    # make bkg.dpi
    # this one uses a off event time size to make a background dpi
    subprocess.run(['batbinevt '  + str(j)
                + ' outfile =bkg.dpi energybins=' +str(energy_bins)
                + ' clobber= ' + str(clobber) + ' outtype=dpi detmask=frb.mask timedel = 0'
                + ' tstart=639072334.000000 tstop=639072896.800600' ],
               shell=True, cwd=results_dir)

    # make sky image
    subprocess.run(['batfftimage frb2.dpi attitude = sw' + str(source_id) + 'sat.fits.gz detmask=frb.mask'
                   + ' outfile = frb.sky bkgfile=bkg.dpi clobber =' + str(clobber)],
                   shell=True, cwd = results_dir)

    # run heasoft source finder
    subprocess.run(['batcelldetect frb.sky outfile=cat.fits'], shell=True, cwd = results_dir)



    return
#-----------------------------------------------------------------------------#
def run_search():
    """ runs search over all bat files in data dir that correspond to FRBS with
    localizations"""
    swift_ids = get_swift_ids_data(data_dir)
    source_ids = swift_ids
    event_dir, hk_dir, results_dir = get_dir_trees(source_ids,data_dir,results)
    for i in range(len(swift_ids)):

        swift_id = swift_ids[i]
        evt_files = get_evt_files(event_dir[swift_id])
        for j in evt_files:
            run_heasoft(swift_id, j, results, event_dir[swift_id],
            hk_dir[swift_id], results_dir[swift_id], clobber=clobber, energy_bins=14-175, time_res=1)


#-----------------------------------------------------------------------------#
def main():
    import argparse
    """
    Start the function to perform RM-synthesis if called from the command line.
    """
    # Help string to be shown using the -h option
    descStr = """
    to be written later"""


    epilog_text="""
    to be written later
    """
    # Parse the command line options
    parser = argparse.ArgumentParser(description=descStr,epilog=epilog_text,
                                 formatter_class=argparse.RawTextHelpFormatter)

    # add arguments like this
    #parser.add_argument("-U", dest="units", type=str, default="Jy/beam",
        #                help="Intensity units of the data. [Jy/beam]")
    #units          = args.units
    args = parser.parse_args()

    #run_search()




In [2]:
# we want to open our .tsv and read out the swift ids that correspond to FRB localizations
# and make our swift_id list from that, maybe even have the option to redownload the data?



def get_swift_ids_data(data_dir):
    '''make list of swift IDS from the data dir'''
    output = subprocess.check_output(["ls",],universal_newlines=True, cwd= data_dir)
    source_ids = output.split() # list of strings the source IDs

    # pop off these dirs if they exist in the list
    source_ids.remove('GUANO')
    source_ids.remove('dump')
    source_ids.remove('inventory')
    source_ids.remove('-')
    source_ids.remove('GUANO')
    source_ids.remove('triggers.tsv')
    source_ids.remove('skycoords.dat')
    return source_ids
    

In [3]:
data_path = '../data'
guano_dump = 'GUANO dump inventory - GUANO triggers.tsv'
guano_path = data_path + '/' + guano_dump

gauno_dump_data = np.genfromtxt(guano_path,delimiter='\t', skip_header=1, dtype= None) # did not skip header
#ids = gauno_dump_data[:,0] # missing leading zeros! 

#tsar_bb = gauno_dump_data[:,-1]

/tmp/ipykernel_5380/2611413336.py:5: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  gauno_dump_data = np.genfromtxt(guano_path,delimiter='\t', skip_header=1, dtype= None) # did not skip header


In [4]:
# function to read relevent info from guano dump inventory

# expecting a .tsv file!
# we want swift IDS, 
# chime_trigger_time
# Tsar good, has BB
# Chime event ID - will make a results file with a summary for each and then an overall results



In [5]:
#swift ID,Trigger time,CHIME id, Tsar ratings, S/N,DM,Baseband processing status,nfreq, Tsar good has BB, Tsar good, no BB

In [83]:

def get_gauno_info(data_dir):
    
    guano_dump = 'GUANO dump inventory - GUANO triggers.tsv'
    guano_path = data_path + '/' + guano_dump
    swift_ids = np.zeros(len(gauno_dump_data))
    chime_trigger = []
    chime_ids = []
    tsar_bb = []
    index_del = []

    # we want the ones with tsar_good and bb == True right? 
    for i in range(len(gauno_dump_data)):
        swift_ids[i] = gauno_dump_data[i][0]
        
        tsar_bb.append(gauno_dump_data[i][-2]) # this is a b'string' format
        if tsar_bb[i] == b'TRUE':
            tsar_bb[i] = True
            chime_trigger.append(gauno_dump_data[i][1]) # this is a b'string' format 
            chime_ids.append(gauno_dump_data[i][2]) # this is a b'string' format
        elif tsar_bb[i] == b'FALSE':
            tsar_bb[i] = False
        else:
            index_del.append(i)
        # I assume this stays in order but gauno_dump format should change
    tsar_arr = np.asarray(tsar_bb[0:min(index_del)])
    index_arr = np.where(tsar_arr == True)
    swift_ids = swift_ids[index_arr[0]]
    return swift_ids, tsar_arr, chime_ids, chime_trigger



In [7]:
#tsar_arr = np.asarray(tsar_bb)

In [28]:
swift_ids[0]

14807020.0

In [85]:
for i in range(len(swift_ids)):
    id_len = len(str(int(swift_ids[i])))
    swift_ids[i] = int('0' * (11-id_len) + str(int(swift_ids[i])))

In [84]:
swift_ids, tsar_arr, chime_ids, chime_trigger = get_gauno_info(data_dir)

In [72]:
chime_trigger[0]

b'2022-01-05 12:24:00'

In [16]:
# need to convert chime_trigger to be in MET for swift (in seconds)
# read evt file time ? 

# center trigger with range (taken as arguments)
# similar method for the background stops 20s before and is all the evt data before

# need to pop open image and make LC, estimate SNR? 
# need to cross ref cat with ra,dec and min, max time resolution
# make a json for key results IE best SNR, with correspondong time window info and RA, DEC and report heafsoft
# with my own 

from swifttools.swift_too import Clock, ObsQuery, VisQuery
from datetime import datetime,timedelta

In [17]:
cc = Clock(swifttime='2022-01-01 00:00:00')
cc

662688000.0,2022-01-01 00:00:00,2021-12-31 23:59:31.643287,-28.356713


In [32]:
cc = Clock(utctime = b'2022-01-05 12:24:00'.decode("utf-8") )

In [33]:
cc

663078268.383033,2022-01-05 12:24:28.383033,2022-01-05 12:24:00,-28.383033


In [22]:
cc.mettime

663078268.383033

In [79]:
from tqdm import tqdm
# make dict of MET_chime_trigger

def get_swift_time_from_chime(chime_trigger, swift_ids):
    MET_chime_trigger = {}
    for i in range(len(chime_trigger)):
        swift_id = swift_ids[i]
        #chime_trigger[i] = chime_trigger[i][1:]
        cc = Clock(utctime = chime_trigger[i].decode("utf-8") )
        MET_chime_trigger[swift_id] = cc.mettime
        
    return MET_chime_trigger

In [ ]:
import time

start = time.time()
print("hello")
end = time.time()
print(end - start)

In [93]:
start = time.time()
test = get_swift_time_from_chime(chime_trigger, swift_ids)
end = time.time()
print(end - start)

KeyboardInterrupt: 

In [92]:
start = time.time()
test2 = get_swift_time_from_chime2(chime_trigger, swift_ids)
end = time.time()
print(end - start)

0.47190332412719727


In [87]:
# next method of geting chime triger time

def get_swift_time_from_chime(chime_trigger, swift_ids):
    chime_trigger_str =[]
    for i in range(len(chime_trigger)):
        chime_trigger_str.append(chime_trigger[i].decode('utf-8'))
    trigger = chime_trigger_str
    cc = Clock()
    cc.utctime = trigger
    cc.submit()
    cc.met
    res = dict(zip(swift_ids, cc.met))
    return res

In [60]:
cc = Clock()
cc.utctime = trigger[0:-40]
cc.submit()
#cc.met

True

In [49]:
trigger = chime_trigger_str

In [59]:
test == test2

NameError: name 'test' is not defined

In [81]:
range(5)

range(0, 5)

In [82]:
for i in range(5):
    print(i)

0
1
2
3
4


In [90]:
# making basic time window for search

# we have chime_trigger which is a dict of [swift_id:swift_met_trigger]

# lets try a basic bcgk window = [0,-20]s b4 trigger and evnt window is 3s

evt_start = chime_trigger[swift_id] - 1.5
evt_end = chime_trigger[swift_id] + 1.5

bck_start = 0
bck_end = chime_trigger[swift_id] - 20

True

In [98]:
evt_start = float(test2[swift_id]) - 1.5

In [95]:
swift_id

95944003.0

In [99]:
evt_start

636025856.073647

In [ ]:
chime_